# Configuration

In [1]:
%load_ext literary.notebook

In [2]:
import typing as tp
from functools import lru_cache
from pathlib import Path

from traitlets.config import (
    Config,
    ConfigFileNotFound,
    JSONFileConfigLoader,
    PyFileConfigLoader,
)

from .project import ProjectOperator

The Literary project uses a well-known file to configure the various components. We can implement the configuration in Python or JSON, so we search for any configuration file with the appropriate stem.

In [3]:
CONFIG_FILE_STEM = "literary_config"

To find the config file, we simply perform a depth-first search of each given path, and return the first file with the appropriate stem.

In [4]:
@lru_cache()
def find_project_config(path, *additional_paths) -> Path:
    """Load the configuration for the current Literary project.

    :param search_paths: starting search paths
    :return:
    """
    visited = set()

    for top_level_path in [path, *additional_paths]:
        for search_path in (top_level_path, *top_level_path.parents):
            # Avoid re-visiting paths
            if search_path in visited:
                break
            visited.add(search_path)

            # Look for any config file
            for p in search_path.glob(f"{CONFIG_FILE_STEM}.*"):
                return p

    raise FileNotFoundError("Couldn't find config file")

Upon finding a configuration file, we can attempt to load it with the known loader classes.

In [5]:
def load_project_config(path: Path) -> Config:
    """Load a project configuration file

    :param path: configuration file path
    :return:
    """
    for loader_cls in JSONFileConfigLoader, PyFileConfigLoader:
        loader = loader_cls(path.name, str(path.parent))
        try:
            config = loader.load_config()
            break
        except ConfigFileNotFound:
            continue
    else:
        raise ValueError(f"{path!r} was not a recognised config file")

    # Add the path to the current configuration file
    config.merge(Config({ProjectOperator.__name__: {"project_path": path.parent.absolute()}}))
    return config